In [ ]:
tool_outputs [{'tool_call_id': 'call_gEcBlLIMiT0vj9R6z1cSBfF7', 'output': 'Tool function understandFile for tool_id call_gEcBlLIMiT0vj9R6z1cSBfF7 is not callable with arguments {"file_name":"data_description.txt","things_to_look_for":"The data features and their descriptions."}.'}]

In [7]:
def understand_file(file_name, things_to_look_for, work_dir = ".", **kwargs):

    lines = read_file(file_name, work_dir = work_dir, **kwargs).split("\n")
    # group lines to blocks so that each block has at most 10000 characters
    counter = 0
    blocks = []
    while counter < len(lines):
        block = []
        start_line_number = counter + 1
        while counter < len(lines) and len("\n".join(block)) + len(lines[counter]) < 10000:
            block.append(lines[counter])
            counter += 1
        if len(block) > 0:
            end_line_number = counter 
            blocks.append(("\n".join(block), start_line_number, end_line_number))
        else:
            end_line_number = start_line_number
            # probably a file of one/few very long line; split by 10000 characters
            for i in range(0, len(lines[counter]), 10000):
                blocks.append((lines[counter][i:i+10000], start_line_number, end_line_number))
            counter += 1

    descriptions  = []
    for idx, (b, start_line_number, end_line_number) in enumerate(blocks):
        start_char_number = sum([len(b) for b in blocks[:idx]])
        end_char_number = start_line_number + len(b)
        prompt = f"""Given this (partial) file from line {start_line_number} character {start_char_number} to line {end_line_number} character {end_char_number}: 
    ``` 
    {b}
    ```
    Here is a detailed description on what to look for and what should returned: {things_to_look_for}
    The description should short and also reference crtical lines in the script relevant to what is being looked for. Only describe what is objectively confirmed by the file content. Do not include guessed numbers. If you cannot find the answer to certain parts of the request, you should say "In this segment, I cannot find ...".
    """

        completion = complete_text_fast(prompt, log_file=kwargs["log_file"]+f"_{idx}")
        descriptions.append(completion)
    if len(descriptions) == 1:
        return descriptions[0]
    else:
        descriptions = "\n\n".join(["Segment {idx}: \n\n" + s for s in descriptions])
        prompt = f"""Given the relevant observations for each segments of a file, summarize to get a cohesive description of the entire file on what to look for and what should returned: {things_to_look_for}
    {descriptions}
    """

        completion = complete_text_fast(prompt, log_file=kwargs["log_file"])

        return completion

In [8]:
tool_outputs = [{'tool_call_id': 'call_gEcBlLIMiT0vj9R6z1cSBfF7', 'output': 'Tool function understandFile for tool_id call_gEcBlLIMiT0vj9R6z1cSBfF7 is not callable with arguments {"file_name":"data_description.txt","things_to_look_for":"The data features and their descriptions."}.'}]

In [9]:
understand_file("data_description.txt", "The data features and their descriptions.")

NameError: name 'read_file' is not defined